# PROJETO - EXTRAÇÃO DE DADOS DO GOOGLE ANALYTICS 4 


### OBJETIVO:

COLETAR DADOS DO GOOGLE ANALYTICS ATRAVÉS DA API DO GOOGLE E DISPONIBILIZAR OS DADOS PARA SEREM CARREGADOS PARA UM BANCO DE DADOS E POSTERIORMENTE GERAR UM DASHBOARD PARA O SETOR DE MARKETING.


### DESCRIÇÃO DO PROCESSO:

- CRIAR UMA API DENTRO DO SITE https://developers.google.com/
- CRIAR CÓDIGO DE AUTENTICAÇÃO E RENOVAÇÃO DE TOKEN OAUTH2.0
- CRIAR CÓDIGO PARA A COLETA DE INFORMAÇÕES DE MÉTRICAS E DIMENSÕES SOLICITADAS PELO SETOR DE MARKETING
- INSERIR OS DADOS COLETADOS EM UM BANCO DE DADOS
- GERAR DASHBOARD DE INDICADORES PARA O SETOR DE MARKETING UTILIZANDO POWER BI.


### OBSERVAÇÕES IMPORTANTES:

POR SE TRATAR DE UM PROJETO REAL, AS ETAPAS DE INSERÇÃO DOS DADOS COLETADOS PELA API NUM BANCO DE DADOS E A GERAÇÃO DO DASHBOARD FORAM REMOVIDAS.

DADOS DE AUTENTICAÇÃO DO CLIENTE TAMBÉM FORAM SUPRIMIDOS DESTE PROJETO.

## 1 - IMPORTAÇÃO DOS PACOTES NECESSÁRIOS

In [ ]:
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import pandas as pd
from collections import defaultdict
from datetime import date
import openpyxl

## 2 - PARÂMETROS GLOBAIS

In [ ]:
scopes = ['https://www.googleapis.com/auth/analytics']    
property_id = 'properties/insira seu ID de Propriedade'
start_date = str(date.today())
end_date = start_date

# Dimensões e métricas gerais para alimentar a tabela tb_users
dimensions_users = ['date']
metrics_users = ['activeUsers','sessions','sessionsPerUser','newUsers','screenPageViews','screenPageViewsPerSession',
                'bounceRate','averageSessionDuration','engagedSessions','active1DayUsers']


# Dimensões e métricas para alimentar a tabela tb_sessions               
dimensions_sessions = ['date','pageTitle']
metrics_sessions = ['screenPageViewsPerSession','sessions']

# Dimensões e métricas para alimentar a tabela tb_geo
dimensions_geo = ['date','region','country','countryId']
metrics_geo = ['sessions']
                
# Dimensões e métricas para alimentar a tabela tb_devices
dimensions_devices = ['date','deviceCategory'] 
metrics_devices = ['sessions']

# Dimensões e métricas para alimentar a tabela tb_channel
dimensions_channels = ['date','sessionDefaultChannelGrouping']
metrics_channels = ['sessions'] 

## 3 - FUNÇÕES

In [ ]:
## AUTENTICAÇÃO 
def ga_auth(scopes):
    creds = None
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', scopes)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', scopes)
            creds = flow.run_local_server(port=0)
        # SALVANDO AS CREDENCIAIS
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('analyticsdata', 'v1beta', credentials=creds)

    return service

## EXTRAÇÃO DE DADOS
def data_extraction(response,dimensions,metrics):
    report_data = defaultdict(list)
    for report in response.get('reports', []):
        rows = report.get('rows', [])
        for row in rows:
            for i, key in enumerate(dimensions):
                report_data[key].append(row.get('dimensionValues', [])[i]['value'])  
            for i, key in enumerate(metrics):
                report_data[key].append(row.get('metricValues', [])[i]['value'])
    return report_data

## RESQUEST API
def requests(dimensions,metrics):
    request = {
      'requests': [
        {
          'dateRanges': [
            {
              'startDate': start_date,
              'endDate': end_date
            }
          ],
          'dimensions': [{'name': name} for name in dimensions],
          'metrics': [{'name': name} for name in metrics],
          'limit': 100000
        }
      ]
    }
    
    return request

## 4 - AUTENTICAÇÃO

In [ ]:
service = ga_auth(scopes)
analytics = ga_auth(scopes)

## 5 - EXTRAINDO OS DADOS DO GA4

In [ ]:
request_users = requests(dimensions_users, metrics_users)
request_sessions = requests(dimensions_sessions, metrics_sessions)
request_geo = requests(dimensions_geo, metrics_geo)
request_devices = requests(dimensions_devices, metrics_devices)
request_channels = requests(dimensions_channels, metrics_channels)

response_users = analytics.properties().batchRunReports(property=property_id, body=request_users).execute()
response_sessions = analytics.properties().batchRunReports(property=property_id, body=request_sessions).execute()
response_geo = analytics.properties().batchRunReports(property=property_id, body=request_geo).execute()
response_devices = analytics.properties().batchRunReports(property=property_id, body=request_devices).execute()
response_channels = analytics.properties().batchRunReports(property=property_id, body=request_channels).execute()


df_users = pd.DataFrame(data_extraction(response_users,dimensions_users,metrics_users)) 
df_sessions = pd.DataFrame(data_extraction(response_sessions,dimensions_sessions,metrics_sessions)) 
df_geo = pd.DataFrame(data_extraction(response_geo,dimensions_geo,metrics_geo)) 
df_devices = pd.DataFrame(data_extraction(response_devices,dimensions_devices,metrics_devices)) 
df_channels = pd.DataFrame(data_extraction(response_channels,dimensions_channels,metrics_channels)) 

## 6 - SALVANDO OS DADOS

In [ ]:
df_users.to_csv('report_users.csv',encoding='utf-8',index=False)
df_sessions.to_csv('report_sessions.csv',encoding='utf-8',index=False)
df_geo.to_csv('report_geo.csv',encoding='utf-8',index=False)
df_devices.to_csv('report_devices.csv',encoding='utf-8',index=False)
df_channels.to_csv('report_channels.csv',encoding='utf-8',index=False)

## 7 - REQUIREMENTS.TXT

In [4]:
!pip3 freeze > requirements.txt